In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel("/kaggle/input/casestudydataset/Talent_Academy_Case_DT_2025.xlsx")

In [ ]:
df.head()

In [ ]:
print(f"Veri seti boyutu: {df.shape[0]} satır, {df.shape[1]} sütun.")
df.info()

In [ ]:
print(df.describe())
print(df.describe(include=['object']).T)

# İlk Adım

### 1- Kolon isimleri güncellenecek
### 2- "Seans" ve "Dakika" ifadeleri satırardan silinecek
### 3- Tamamen aynı olan satırlar silinecek (ps: Normal koşullarda aynı hasta aynı hastalık için aynı tedaviyi almaya defalarca gelebilir ancak burda bu durumu kıyaslayabileceğimiz bir tarih verisi olmadığı için kayıtlı satırları duplice olarak değerlendiriyoruz)
### 4- tüm string değerler lower olacak
### 5- Eksik veri olan satırlara "nan" değer atanacak

In [ ]:
#1- kolon isimleri daha uygun olmalı

df = df.rename(columns={
    "HastaNo": "hasta_no",
    "Yas": "yas",
    "Cinsiyet": "cinsiyet",
    "KanGrubu": "kan_grubu",
    "Uyruk": "uyruk",
    "KronikHastalik": "kronik_hastalik",
    "Bolum": "bolum",
    "Alerji": "alerji",
    "Tanilar": "tanilar",
    "TedaviAdi": "tedavi_adi",
    "TedaviSuresi": "tedavi_seans_sayisi",
    "UygulamaYerleri": "uygulama_yeri",
    "UygulamaSuresi": "seans_suresi_dk"
})

In [ ]:
# 2- "Dakika" ve "Seans" ifadelerini temizlemek 

df["tedavi_seans_sayisi"] = df["tedavi_seans_sayisi"].str.replace("Seans", "", regex=False)
df["seans_suresi_dk"] = df["seans_suresi_dk"].str.replace("Dakika", "", regex=False)

# 2) Sağlı sollu boşlukları temizle
df["tedavi_seans_sayisi"] = df["tedavi_seans_sayisi"].str.strip()
df["seans_suresi_dk"] = df["seans_suresi_dk"].str.strip()

# 3) Integer’a çevir
df["tedavi_seans_sayisi"] = df["tedavi_seans_sayisi"].astype(int)
df["seans_suresi_dk"] = df["seans_suresi_dk"].astype(int)

In [ ]:
#3- Duplice satır var mı varsa sil
# 1. Tüm satırlara göre duplicate olanları işaretle
mask = df.duplicated()

# 2. Kaç tane duplicate var (tamamen aynı olan satır sayısı)
duplicate_count = mask.sum()
print("Tamamen aynı olan satır sayısı:", duplicate_count)

# 3. Duplice satırları sil
df_unique = df.drop_duplicates()

In [ ]:
df_unique.shape[0] + 928

In [ ]:
#Kategorik kolonları seç

cat_cols = [
    "cinsiyet",
    "kan_grubu",
    "uyruk",
    "kronik_hastalik",
    "bolum",
    "alerji",
    "tanilar",
    "tedavi_adi",
    "uygulama_yeri"
]

In [ ]:
#Küçük harfe dönüşümü
for col in cat_cols:
    df_unique.loc[:, col] = df_unique[col].str.lower()

# Kategorik kolonları gözlemleme

In [ ]:
print("--- Kategorik Kolonların Benzersiz Değerleri ---")
for col in cat_cols:
    print(f"--- {col} örnekleri---")
    
    unique_values = df_unique[col].unique()
    print(unique_values[:20])
    
    print(f"Toplam unique: {df_unique[col].nunique()}")
    print("-" * 20)

In [ ]:


cinsiyet_dagilimi = df_unique['cinsiyet'].value_counts()
kan_grubu_dagilimi = df_unique['kan_grubu'].value_counts()
uyruk_dagilimi = df_unique['uyruk'].value_counts()

sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'DejaVu Sans'

# Grafik 1: Cinsiyet Dağılımı
plt.figure(figsize=(8, 6)) 
sns.barplot(x=cinsiyet_dagilimi.index, y=cinsiyet_dagilimi.values, palette='viridis')
plt.title('Cinsiyet Dağılımı', fontsize=16)
plt.xlabel('Cinsiyet', fontsize=12)
plt.ylabel('Kişi Sayısı', fontsize=12)

plt.show()

# Grafik 2: Kan Grubu Dağılımı
plt.figure(figsize=(12, 7)) # Grafik boyutunu ayarla
sns.barplot(x=kan_grubu_dagilimi.index, y=kan_grubu_dagilimi.values, palette='plasma')
plt.title('Kan Grubu Dağılımı', fontsize=16)
plt.xlabel('Kan Grubu', fontsize=12)
plt.ylabel('Kişi Sayısı', fontsize=12)
plt.xticks(rotation=45) 
plt.tight_layout() 
plt.show()

plt.figure(figsize=(12, 7)) # Grafik boyutunu ayarla
sns.barplot(x=uyruk_dagilimi.index, y=uyruk_dagilimi.values, palette='magma')
plt.title('Uyruk Dağılımı', fontsize=16)
plt.xlabel('Uyruk', fontsize=12)
plt.ylabel('Kişi Sayısı', fontsize=12)
plt.xticks(rotation=45) 
plt.tight_layout() 
plt.show()

In [ ]:
df_unique.isnull().sum()

# Karmaşık olmayan kolonlar için Eksik değer yönetimi

In [ ]:
# Alerji ve kronik hastalık kişlerde bulunmayabilir. Daha sonra kronik hastalık konusunda daha derin irdeleme yaparız.

df_unique = df_unique.fillna({
    'alerji': 'Yok',
    'kronik_hastalik': 'Yok',
})

In [ ]:
#Normal koşullarda mod değer ile doldurulabilirdi ama bilmeme durumunu korumak ve belirtmek istedim
# Cinsiyet ve Kan grubu kolonundaki NaN değerleri 'Bilinmiyor' kategorisi ile dolduruluyot
df_unique['cinsiyet'].fillna('Bilinmiyor', inplace=True)

df_unique['kan_grubu'].fillna('Bilinmiyor', inplace=True)

In [ ]:
df_unique.replace(r'^\s*$', np.nan, regex=True)

# numerik kolonları gözlemleme

## aykırı değer
## veri dağılımı
## korelasyon

In [ ]:
print(df_unique.describe())

In [ ]:
num_cols = df_unique.select_dtypes(include=['int64', 'float64']).columns
print("Sayısal kolonlar:", num_cols)

In [ ]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

plt.figure()
sns.histplot(df['yas'], kde=True, bins=10)
plt.title('Yaş Kolonunun Dağılımı (Histogram & KDE)')
plt.xlabel('Yaş')
plt.ylabel('Frekans (Kişi Sayısı)')
plt.show()

plt.figure()
sns.boxplot(x=df['yas'])
plt.title('Yaş Kolonundaki Aykırı Değerler (Box Plot)')
plt.xlabel('Yaş')
plt.show()

plt.figure()
sns.scatterplot(x='yas', y='tedavi_seans_sayisi', data=df)
plt.title('Yaş ve Tedavi Seans Sayısı Arasındaki İlişki (Scatter Plot)')
plt.xlabel('Yaş')
plt.ylabel('Tedavi Seans Sayısı')
plt.show()

# Tüm Değişkenler Arasındaki İlişki: Korelasyon Isı Haritası
plt.figure()
correlation_matrix = df.corr(numeric_only=True)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Nümerik Kolonların Korelasyon Matrisi (Heatmap)')
plt.show()

In [ ]:
# İki grafiği yan yana çizdirmek için bir figür ve eksenler oluşturalım
fig, axes = plt.subplots(1, 2)

fig.suptitle('Tedavi Seans Sayısı ve Süresi İçin Aykırı Değer Analizi', fontsize=16)

# Grafik 1: Tedavi Seans Sayısı
sns.boxplot(x=df['tedavi_seans_sayisi'], ax=axes[0])
axes[0].set_title('Tedavi Seans Sayısı Aykırı Değerleri')
axes[0].set_xlabel('Seans Sayısı')

# Grafik 2: Seans Süresi (dk)
sns.boxplot(x=df['seans_suresi_dk'], ax=axes[1])
axes[1].set_title('Seans Süresi (dk) Aykırı Değerleri')
axes[1].set_xlabel('Süre (Dakika)')

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Başlığın sıkışmasını engelle
plt.show()

# Veri Temizliği

Bu noktada yazım hataları, mantıksal olarak hatalı bilgiler gibi süreci etkileyecek veriler temizlenip düzenlenecektir.

In [ ]:
#uyruk kolonunda türkiye dışındakilerin sayısı az bu sebeple türkiye ve diğerleri olarak ayıralım
# Türkiye dışındakileri 'Diğer' yap
df_unique["uyruk_cleaned"] = df_unique["uyruk"].apply(lambda x: "türkiye" if x == "türkiye" else "diğer")

# Bolum Kolonu Veri temizliği

In [ ]:

print("--- bolum ---")
print(df_unique["bolum"].value_counts(dropna=False))

In [ ]:
#kolonun ihtiyaç duyduğu düzenlemeleri keşfetmek amacıyla
benzersiz_bolumler = df_unique['bolum'].unique()

print("Bölünmemiş benzersiz bölümler:")
for bolum in benzersiz_bolumler:
    print(bolum)

In [ ]:
mapping = {
    'i̇ç hastalıkları': 'iç hastalıkları',
    'laboratuar': np.nan,
    'fiziksel tıp ve rehabilitasyon,solunum merkezi': 'fiziksel tıp ve rehabilitasyon',
    # kalp-damar cerrahisi ve kardiyoloji birleştirilebilir
    #iç hastalıkalrı ve genel cerrahide birleştirilebilir belki ama cerrahi ve non invazif alanların ayrımı önemli olabilir
}

def normalize_bolum(s):
    """
    Belirtilen kurallara göre 'bolum' değerlerini temizler ve standartlaştırır.
    """
    if pd.isna(s) or s is None or s == 'yok':
        return np.nan
        
    s = s.strip().lower()

    return mapping.get(s, s)

df_unique['bolum_cleaned'] = df_unique['bolum'].apply(normalize_bolum)

print(df_unique['bolum_cleaned'].value_counts(dropna=False))

In [ ]:

grup = df_unique.groupby('bolum_cleaned')['seans_suresi_dk']

# Ortalama ve hasta sayısını hesapla
istatistik = grup.agg(
    ortalama_tedavi_suresi='mean',
    hasta_sayisi='count'
).reset_index()

print(istatistik)

# Alerji Kolonu Veri temizliği

In [ ]:
benzersiz_alerjiler = df_unique['alerji'].dropna().unique()

print("Bölünmemiş benzersiz alerji:")
for alerji in benzersiz_alerjiler:
    print(alerji)

In [ ]:

alerjiler = df_unique['alerji'].str.strip()
split_list = []
for item in alerjiler:
    split_list.extend([x.strip().lower() for x in item.split(',')])


unique_allergies = set(split_list)

print("Farklı alerjiler (unique):")
print(unique_allergies)

In [ ]:
# Normalize etmek için mapping
mapping = {
    'gri̇pi̇n': 'gripin',
    'gripin': 'gripin',
    'volteren': 'voltaren', 
    'voltaren': 'voltaren',
    'arveles': 'arveles',
    'novalgin': 'novalgin',
    'coraspin': 'coraspin',
    'polen': 'polen',
    'toz': 'toz',
    'sucuk': 'sucuk',
    'yer fıstığı': 'yer fıstığı',
    'yok': 'yok'
}

# Normalize et
def clean_alerji(item):
    """
    Tek bir alerji değerini temizler ve standartlaştırır.
    """
    if pd.isna(item) or item is None:
        return np.nan
           
    item = str(item).lower().strip()
    
    # Virgülle ayır ve her bir öğeyi temizle
    items = [x.strip() for x in item.split(',') if x.strip()]
    
    # Mapping'i uygula ve benzersiz değerleri al
    normalized_items = [mapping.get(x, x) for x in items]
    
    # Çift değerleri engellemek için set kullan
    return list(set(normalized_items))
                                # liste döndür

In [ ]:
df_unique['alerji_cleaned'] = df_unique['alerji'].apply(clean_alerji)

In [ ]:
# Gruplama (ilaç/gıda/çevresel/yok) ---
groups = {
    "ilac": ["gripin", "voltaren", "arveles", "novalgin", "coraspin"],
    "gida": ["yer fıstığı", "sucuk"],
    "cevresel": ["polen", "toz"],
    "yok": ["yok"]
}

def map_alerji_gruplari(alerji_list):
    if not isinstance(alerji_list, list):
        return []
    yeni_liste = []
    for grup, elemanlar in groups.items():  # <-- burada grup_map değil, groups olmalı
        if any(a in elemanlar for a in alerji_list):
            yeni_liste.append(grup)
    return yeni_liste

df_unique['alerji_grup'] = df_unique['alerji_cleaned'].apply(map_alerji_gruplari)

In [ ]:
df_unique[['alerji_cleaned','alerji_grup']].head()

# Kronik Hastalıklar

In [ ]:
unique_diseases = set()
for row in df_unique["kronik_hastalik"].dropna():
    diseases = [d.strip() for d in row.split(",")]
    unique_diseases.update(diseases)

# Tekil hastalıkları alfabetik sırala
unique_diseases = sorted(unique_diseases)

# Sonucu yazdır
for i, d in enumerate(unique_diseases, 1):
    print(f"{i}. {d}")

In [ ]:
#kas hastalıklarını bilerek gruplamıyorum tedavi süresi üzerinde etkisi diğerlerine göre daha yüksek
def clean_chronic_diseases(data):
    cleaned_data = []
    typo_map = {
        "hiportiroidizm": "tiroid hastalıkları",
        "hipotirodizm": "tiroid hastalıkları",
        "hipotirodism": "tiroid hastalıkları",
        "hipertiroidizm":"tiroid hastalıkları",
        "guatr": "tiroid hastalıkları",
        "aritmi": "kardiyovasküler hastalıklar",
        "hipertansiyon": "kardiyovasküler hastalıklar",
        "kalp yetmezliği": "kardiyovasküler hastalıklar", 
        "Yok":"kronik hastalık yok"
        
    }

    for item in data:
        if not isinstance(item, str):
            cleaned_data.append([])
            continue

        # Küçük harfe çevirme ve baş/son boşlukları silme
        cleaned_item = item.lower().strip().replace('  ', ' ')
        
        # Virgüllerle ayırma
        diseases = [d.strip() for d in cleaned_item.split(',') if d.strip()]
        
        # Yazım hatalarını düzeltme
        corrected_diseases = []
        for disease in diseases:
            disease = typo_map.get(disease, disease)  # yanlış yazımı düzelt
            if disease not in corrected_diseases:     # aynı satırda tekrar varsa atla
                corrected_diseases.append(disease)

        cleaned_data.append(corrected_diseases)
    
    return cleaned_data


cleaned_list = clean_chronic_diseases(df_unique["kronik_hastalik"])

all_diseases = [d for sublist in cleaned_list for d in sublist]
unique_counts = pd.Series(all_diseases).value_counts()

print("Temizlenmiş unique hastalıklar ve frekansları:\n")
print(unique_counts)
print(f"\nToplam farklı hastalık sayısı: {unique_counts.shape[0]}")

In [ ]:
df_unique["kronik_hastalik_cleaned"] = clean_chronic_diseases(df_unique["kronik_hastalik"])

print(df_unique[["kronik_hastalik","kronik_hastalik_cleaned"]].head(20))

# Uygulama yeri

In [ ]:
benzersiz_uygulama_yeri = df_unique['uygulama_yeri'].dropna().unique()

print("Bölünmemiş benzersiz uygulama_yeri:")
for uygulama_yeri in benzersiz_uygulama_yeri:
    print(uygulama_yeri)

In [ ]:
def clean_treatment_area(s):
    """
    Bu fonksiyon bir string'i temizler, virgüllerle ayırır ve her bir öğeyi düzeltir.
    """
    if pd.isna(s):
        return []
    
    s = str(s).lower().replace(' bölgesi', '').replace('bilek', 'bileği')

    return [item.strip() for item in s.split(',')]

df_unique['uygulama_yeri_cleaned'] = df_unique['uygulama_yeri'].apply(clean_treatment_area)

tum_bolgeler = [item for sublist in df_unique['uygulama_yeri_cleaned'] for item in sublist]

benzersiz_bolgeler = sorted(list(set(tum_bolgeler)))

print("Temizlenmiş, ayrılmış benzersiz uygulama yerleri:\n")
for bolge in benzersiz_bolgeler:
    print(bolge)

In [ ]:
df_unique.head()

# Tedavi Adı

In [ ]:
benzersiz_tedaviler = df_unique['tedavi_adi'].unique()

# Alfabetik sırala
benzersiz_tedaviler = sorted(benzersiz_tedaviler)

print("Bölünmemiş benzersiz tedaviler:")
# Listenin her öğesini numaralandırarak yazdır
for i, tedavi in enumerate(benzersiz_tedaviler, 1):
    print(f"{i}. {tedavi}")

In [ ]:
import re
import unicodedata

def _process_single_phrase(phrase):
    """
    Bu yardımcı fonksiyon, '+' ile ayrılmış TEK bir bölümü işler.
    Yön ve evre bilgilerini bulur, standartlaştırır ve birleştirir.
    """
    processed_name = phrase.strip()
    side_suffix = ''
    phase_suffix = ''

    # Evre bilgisini bul ve ayır
    phase_match = re.search(r'(\(evre \d+\))|([- ](\d+))$', processed_name)
    if phase_match:
        if phase_match.group(1):
            # Mevcut parantezli yapıyı alırken başına boşluk ekle
            phase_suffix = f" {phase_match.group(1)}"
        elif phase_match.group(2):
            # Yeni parantezli yapı oluştururken başına boşluk ekle
            num = phase_match.group(3)
            phase_suffix = f" (evre {num})"
        processed_name = processed_name.replace(phase_match.group(0), '')

    # Yön bilgisini bul ve ayır
    if 'bialetarl' in processed_name or 'bilarteral' in processed_name or 'bilateral' in processed_name:
        side_suffix = ' (bilateral)'
        processed_name = re.sub(r'\b(bialetarl|bilarteral|bilateral)\b', '', processed_name)
    else:
        side_match = re.search(r'\b(sağ|sol)\b', processed_name)
        if side_match:
            side = side_match.group(1)
            side_suffix = f' ({side})'
            processed_name = re.sub(r'\s*\b(sağ|sol)\b\s*', ' ', processed_name)

    # Son temizlik
    processed_name = processed_name.strip('-,. ')
    processed_name = re.sub(r'\s+', ' ', processed_name).strip()

    return f"{processed_name}{phase_suffix.strip()}{side_suffix}"


def standardize_treatment_name(name):
    """
    Bu ana fonksiyon, 'sağ+sol' gibi özel durumları öncelikli olarak ele alır,
    ardından metni '+' karakterine göre bölümlere ayırır.
    """
    if not isinstance(name, str):
        name = str(name)
    
    # Genel Karakter Normalizasyonu
    processed_name = unicodedata.normalize('NFKC', name)
    processed_name = processed_name.replace('i̇', 'i').lower() 
    processed_name = re.sub(r'\s*-\s*', ' - ', processed_name)
    processed_name = re.sub(r'\s+', ' ', processed_name).strip()

    # --- YENİ EKLENEN BÖLÜM: Kısaltmaları ve Sık Yapılan Hataları Düzeltme ---

    # 2. Kelimeye bitişik sayıları ayır (örn: "koksartroz1" -> "koksartroz 1")
    # Bu, sonraki evre tespitinin doğru çalışmasını sağlar.
    processed_name = re.sub(r'omuz\s+(impingement sendromu|impingement|impingemen)', 'impingement (omuz)', processed_name)
    
    processed_name = re.sub(r'([a-zA-ZıİğĞüÜşŞöÖçÇ])(\d+)\b', r'\1 \2', processed_name)

    if processed_name == 'bel':
        return 'bel rehabilitasyonu'

    

    if processed_name in ['deneme', 'onur', 'xx']:
        return 'nan'
    
    # 3. Kapsamlı değiştirme listesi
    replacements = {
        # Yazım Hataları (Typo)
        "ağrsıı": "ağrısı", "ağrisi": "ağrısı", "impimgement": "impingement", "myodascial":"myofascial",
        "impingemen ": "impingement ", "kapsülüt": "kapsülit", "muskuler": "muscular","boz ": "bozukluğu",
        "send": "sendromu","geç dönme":"geç dönem", "op.": "operasyonu", "uş": "uç", " ravmatik": " travmatik", "quadriseps": "kuadriseps", "palntar": "plantar",
        "el rehabilitasyon programı":"el rehabilitasyonu", "implanti":"protezi",
        "manüpilasyon":"manipülasyon","trapezz":"trapez",
        # Kısaltmalar
        "öçb": "ön çapraz bağ", "ftr": "rehabilitasyonu", "iv disk": "intervertebral disk",
        "cmc": "karpometakarpal", "ptr":"pelvik taban rehabilitasyonu","beyin tm":"beyin tümörü",
        
        # Operasyon terimleri (en spesifik olanlar önce)
        "op lusu": "(post-op)", "post op": "(post-op)", " opl ": "(post-op)"," implantı":" protezi",
        "diz op": "diz operasyonu","postop":"(post-op)",
    }
    for old, new in replacements.items():
        processed_name = processed_name.replace(old, new)

    processed_name = re.sub(r'(ön çapraz bağ)(?! rehabilitasyonu)', r'\1 rehabilitasyonu', processed_name)

    processed_name = processed_name.replace('ataksi reh', 'ataksi rehabilitasyonu')
    processed_name = processed_name.replace('femur uzatma op', 'femur uzatma operasyonu')
    processed_name = processed_name.replace('diz eklem protezi', 'diz protezi')
    processed_name = processed_name.replace('(post-op) diz artroskopi', 'diz artroskopi (post-op)')
    processed_name = processed_name.replace('(post-op) kalça protezi', 'kalça protezi (post-op)')
    processed_name = processed_name.replace('aşil tamiri', 'aşil rüptürü operasyonu')
    processed_name = processed_name.replace('menisküs tamiri', 'menisküs operasyonu')
    if processed_name == 'el rehabilitasyonu - el rehabilitasyonu':
        return 'el rehabilitasyonu'
    processed_name = re.sub(r'\bpre\s+(op|operasyonu)\b', '(pre-op)', processed_name)
    processed_name = re.sub(r'\breh\s*$', 'rehabilitasyonu', processed_name)
    processed_name = re.sub(r'\bağrıs\s*$', 'ağrısı', processed_name)
    processed_name = re.sub(r'\breha\s*$', 'rehabilitasyonu', processed_name)
    processed_name = re.sub(r'\brehabilitasyon\s*$', 'rehabilitasyonu', processed_name)
    processed_name = re.sub(r'\bop\s*$', 'operasyonu', processed_name)


    processed_name = processed_name.replace(' implant ', ' protezi') 
    processed_name = processed_name.replace('impingement (', 'impingement sendromu (')

    
    # Bölme işleminden önce 'sağ+ sol' kalıbını kontrol et
    if 'sağ+ sol' in processed_name:
        # 'sağ+ sol' ifadesini metinden temizle
        base_name = processed_name.replace('sağ+ sol', '').strip()
        # Kalan kısmı (örn: "humerus kırığı") normal şekilde işle
        processed_base = _process_single_phrase(base_name)
        # Sonuca '(bilateral)' ekle ve büyük harf yap
        return f"{processed_base.capitalize()} (bilateral)"

    # Eğer 'sağ+ sol' yoksa, önceki gibi bölme mantığıyla devam et
    parts = processed_name.split('+')
    processed_parts = []

    for part in parts:
        processed_part = _process_single_phrase(part)
        processed_parts.append(processed_part)


    return ' + '.join(processed_parts)

# Fonksiyonu mevcut 'benzersiz_tedaviler' listenize uygulayın
processed_list = [standardize_treatment_name(t) for t in benzersiz_tedaviler]

# İşlem sonrası aynı hale gelen girdiler olabileceği için listeyi tekrar benzersiz yapıp sıralayın
standartlastirilmis_liste = sorted(list(set(processed_list)))

# Sonucu istenen formatta numaralandırarak yazdırın
print("\nStandartlaştırılmış ve Gruplanmış Benzersiz Tedaviler:")
for i, tedavi in enumerate(standartlastirilmis_liste, 1):
    print(f"{i}. {tedavi}")

In [ ]:
df_unique['tedavi_cleaned'] = df_unique['tedavi_adi'].apply(standardize_treatment_name)

In [ ]:
# Temizleme fonksiyonu bert için
def temizle(metin):
    if not isinstance(metin, str):
        return ""
    metin = metin.lower()
    metin = re.sub(r'\s*\([^)]*\)', '', metin)
    metin = re.sub(r'[\d\.\+]+', '', metin)
    metin = metin.replace('-', ' ') 
    metin = ' '.join(metin.split()) 
    return metin

df_unique['tedavi_cleaned_bert'] = df_unique['tedavi_cleaned'].apply(temizle)
print("Temizlenmiş Tedavi Adları (İlk 5):")
print(df_unique['tedavi_cleaned_bert'].head())

# Tanılar

In [ ]:
benzersiz_tanilar = df_unique['tanilar'].dropna().unique()

print("Bölünmemiş benzersiz tanilar:")
for tani in benzersiz_tanilar:
    print(tani)

In [ ]:
import pandas as pd
import re

def split_and_clean_diagnoses(diagnosis_str):
    """
    Tani dizesini virgüle göre ayirir, her bir tanıyı temizler ve
    bos degerleri atlayarak yeni bir liste olusturur.
    """
    if not isinstance(diagnosis_str, str):
       
        return []
    
    raw_diagnoses = re.split(r'\s*,\s*', diagnosis_str.strip())
  
    cleaned_diagnoses = [diag.strip() for diag in raw_diagnoses if diag.strip()]
    
    return cleaned_diagnoses

In [ ]:
df_unique['ayrılmış_tanılar'] = df_unique['tanilar'].apply(split_and_clean_diagnoses)

In [ ]:
unique_tanilar = set()
for row in df_unique["tanilar"].dropna():
    tanilar = [d.strip() for d in row.split(",")]
    unique_tanilar.update(tanilar)

# Tekil hastalıkları alfabetik sırala
unique_tanilar = sorted(unique_tanilar)

print(unique_tanilar)

In [ ]:
import re
import unicodedata
import pandas as pd

def normalize_spaces(text: str) -> str:
    """Unicode boşlukları (örn. NO-BREAK SPACE, thin space) normal boşluğa çevirir."""
    return ''.join(' ' if unicodedata.category(c) == 'Zs' else c for c in text)

def clean_diagnoses_entry(entry: str) -> list:
    if not isinstance(entry, str) or pd.isna(entry):
        return []

    entry = unicodedata.normalize("NFKC", entry)
    entry = normalize_spaces(entry)
    entry = entry.lower()
    entry = (entry.replace('i̇', 'i')
                   .replace('`', '')
                   .replace('"','')
                   .replace("'",""))r
    entry = (entry.replace("ağrisi", "ağrısı")
                   .replace("eklemde ağrı", "eklem ağrısı")
                   .replace("tanimlanmamiş", "tanımlanmamış").replace("safra yolu taşı","safra taşı"))

    entry = entry.replace("şimdiki", "akut")
    entry = entry.replace("radikulopati", "radikülopati")
    entry = entry.replace("nöralji ve nörit", "nevralji ve nevrit")
    entry = ''.join(ch for ch in entry if not unicodedata.category(ch).startswith('C'))
    entry = re.sub(r'[\u00ad\u200b-\u200f\u202a-\u202e\ufeff\u00a0]', '', entry)
    entry = entry.replace('_x000d_', '')
    entry = re.sub(r',,+', ',', entry)
    entry = re.sub(r'\s*,\s*', ', ', entry)
    entry = re.sub(r'\s+', ' ', entry).strip()
    entry = re.sub(r'\[.*?\]', '', entry)
    diagnoses = [x.strip() for x in entry.split(',') if x.strip()]

    return sorted(set(diagnoses))

In [ ]:
df_unique['tanilar_cleaned'] = df_unique['tanilar'].apply(clean_diagnoses_entry)

In [ ]:
all_diagnoses = [diagnosis for diagnosis_list in df_unique['tanilar_cleaned'].dropna() for diagnosis in diagnosis_list]
unique_diagnoses = sorted(list(set(all_diagnoses)))
print("Tüm benzersiz tanılar:\n") 
for diagnosis in unique_diagnoses: print(diagnosis)

In [ ]:
grup_a_keywords = set(['anoksik', 'ataksi', 'benign', 'beyin', 'beynin', 'distoni', 'equina', 'hemipleji', 'inme', 'hemiparezi', 'kauda', 'malign', 'meningomyelosel', 'merkezi', 'motor', 'multipl', 'neoplazmı', 'nöron', 'palsi', 'parapleji', 'parkinson', 'serebral','serebrovasküler', 'sistemi', 'skleroz', 'tetrapleji'])
grup_b_keywords = set(['akciğer', 'amfizem', 'amputasyon', 'amputasyonlar', 'amyotrofi', 'aort', 'aorta', 'aortik', 'arter', 'arterlerin', 'arterlerinin', 'astım', 'aterosklerotik', 'aterosklerozu','brakeal', 'bronşiektazi', 'bronşit', 'buerger', 'böbrek', 'damar', 'damarlar', 'distrofi', 'doğum','endokrin', 'enfarktüsü', 'enfektif', 'erb','fantom', 'fibrozisli', 'girişim', 'greftleri', 'hidrosefali', 'hipertansif', 'hipertansiyon', 'ilaca', 'implant', 'iskemi', 'kalp', 'kapak', 'kapağı', 'kardiyomiyopati', 'kardiyovasküler', 'klumpke', 'kollaps', 'komplikasyon', 'konjenital','lenf', 'lenfatik', 'lenfödem', 'lumbosakral', 'mediasten', 'mitokondriyal', 'mitral', 'miyokard', 'miyokardiyal', 'miyopati', 'miyopatiler', 'miyotonik', 'musküler', 'nakli', 'nevraljik', 'nodlarının', 'intersitisyel', 'obliterans', 'obstrüktif', 'paralizisi', 'pleksus', 'pnömokonyoz', 'poliomyelit', 'postmastekdomi', 'pulmoner', 'renal', 'renovasküler', 'romatizmal', 'sekonder', 'sessiz', 'solunum', 'trikuspid', 'tromboanjitis', 'tüberküloz','yetmezlik', 'yetmezliği'])
grup_c_keywords = set(['adezif', 'algonörodistrofi', 'alt', 'ankilozan', 'anormallikleri', 'bacağın', 'baldırın', 'bağımlı', 'başparmak', 'başına', 'dijital', 'diplejisi', 'diyabetes','diz protezi', 'dolaşım', 'dönmesi', 'eklem', 'ekstansör', 'ekstremitenin', 'enflamatuvar', 'ezilme', 'fasiyal', 'femur', 'femurun', 'fibula', 'fleksor', 'fleksör', 'fonksiyon', 'gerilmesi', 'göğüs', 'humerus', 'idiopatik', 'ikinci', 'implantları', 'implantı', 'intirinsik', 'kaburganın', 'kalkaneus', 'kalıtsal', 'kapsüliti', 'kemeri', 'kemik', 'klavikula', 'kol', 'komplikasyonuyla', 'korozyonlar', 'kuf', 'kırık', 'kırıkları', 'kırığı', 'lateral', 'ligamentlerinin', 'malleol', 'medial', 'median', 'mellitüs', 'menisküs','meralgia paraestetika','metatarsal', 'mononöropati', 'mononöropatiler', 'mononöropatileri', 'monopleji', 'monoplejisi', 'insülin', 'nöropati', 'omuzun', 'ortopedik', 'paralitik', 'parmak', 'parmağı', 'parmağın', 'parmağının', 'patella', 'periferik', 'pertrokanterik', 'polinöropati', 'polinöropatiler', 'radial', 'radius', 'rotator', 'sendrom', 'sendromlar', 'siniri', 'sinirinin', 'sinirlerin', 'skapula', 'skolyoz', 'spondilit', 'subtrokanterik', 'sınıflanmış', 'talus', 'tarsal', 'tek', 'tendon', 'tendonların', 'tendonunun', 'tibia', 'tibianın', 'tortikollis', 'ucu', 'ulna', 'ulnar', 'uzun', 'uç', 'vertigo', 'vestibüler',  'yanığı','çıkık', 'şaft', 'şaftlarının', 'şaftı'])
grup_d_keywords = set(['dorsalji','bisipital tendinit','bursit','gonartroz','koksartroz','artroz','radikülopati','kalkaneal spur','disk bozuklukları','işemede diğer zorluklar','sinovit','halluks valgus','inkontinans','fibromiyalji','spondilolistezis','romatoid artrit','kifoz','lordoz','sırt ağrısı','bel ağrısı','boyun ağrısı'])


In [ ]:
#verileri var olan kolonlardaki değerlerdn yola çıkarak dolruabildiğimiz verileri dolduruyoruz.
condition = (df_unique['tanilar_cleaned'].str.len() == 0) & (df_unique['tedavi_cleaned'].notna()) & (df_unique['tedavi_cleaned'] != 'nan')

df_unique.loc[condition, 'tanilar_cleaned'] = df_unique.loc[condition, 'tedavi_cleaned'].apply(lambda x: [x])

In [ ]:
def gruplari_bul_nihai(tanilar_list):
    """
    Verilen tanı listesini analiz eder ve eşleşen grupları döndürür.
    Hiçbir gruba uymayanlar 'D Grubu' olarak atanır.
    """
    import numpy as np
    
    if not isinstance(tanilar_list, list) or not tanilar_list:
        return "D Grubu"
    
    birlesik_metin = " ".join(tanilar_list).lower()
    
    bulunan_gruplar = []
    
    for keyword in grup_a_keywords:
        if keyword in birlesik_metin:
            bulunan_gruplar.append('A Grubu')
            break
            
    for keyword in grup_b_keywords:
        if keyword in birlesik_metin:
            bulunan_gruplar.append('B Grubu')
            break
            
    for keyword in grup_c_keywords:
        if keyword in birlesik_metin:
            bulunan_gruplar.append('C Grubu')
            break

    for keyword in grup_d_keywords:
        if keyword in birlesik_metin:
            bulunan_gruplar.append('D Grubu')
            break

    # Hiçbir grup bulunmadıysa D Grubu ata
    if not bulunan_gruplar:
        return "D Grubu"
    else:
        return ", ".join(bulunan_gruplar)

df_unique['SGK_Tani_Grubu'] = df_unique['tanilar_cleaned'].apply(gruplari_bul_nihai)

print(df_unique[['tanilar_cleaned', 'SGK_Tani_Grubu']].head())

In [ ]:
# Grup bazında seans_suresi_dk ortalamasını hesapla
ortalama_seans = df_unique.groupby('SGK_Tani_Grubu')['seans_suresi_dk'].mean().reset_index()

# Sütun adını anlamlı yap
ortalama_seans = ortalama_seans.rename(columns={'seans_suresi_dk': 'ortalama_seans_suresi_dk'})

print(ortalama_seans)

# Uygulama yeri kolonu eksik değer doldurma
Burada elimizdeki bilgiler doğrultusunda bu düzenlemeyi yapmak istiyorum.

In [ ]:
filtre = (df_unique['uygulama_yeri'].fillna('').str.strip() == '')

sonuc_tamami = df_unique.loc[filtre, ['tedavi_cleaned', 'uygulama_yeri']]

pd.set_option('display.max_rows', None)

print("--- 'uygulama_yeri' Kolonu Boş Olan Satırların Tamamı (Kısaltma Olmadan) ---")
print(sonuc_tamami)

pd.reset_option('display.max_rows')

In [ ]:
KEYWORD_MAP  = {
    # --- Kesin Olarak Eşleştirilenler ---
    'Humerus kırığı': 'omuz',
    'aşil rüptürü operasyonu': 'ayak bileği',
    'alt ekstremite atrofi': 'kalça, diz, ayak bileği',
    'alt ekstremite atrofi + yürüme': 'tüm vücut',
    'beyin kanaması + atrofi': 'tüm vücut',
    'boyun - düşük kol': 'boyun', #düşük kol diye bir hastalık yok check edilecek
    'diz menisküs': 'diz',
    'diz menisküs erken mobilizasyon': 'diz',
    'diz operasyonu - erken rehabilitasyonu': 'diz',
    'diz protezi': 'diz',
    'diz protezi sonrasi tam yük': 'diz',
    'diz protezi - erken rehabilitasyonu': 'diz',
    'dorsalji': 'sırt',
    'dorsalji - bel + eklem ağrısı': 'bel',
    'dorsalji + servikal myelomalazi': 'boyun',
    'femur uzatma operasyonu': 'kalça',
    'hemartrotik diz': 'diz',
    'hemartrotik diz - kas kuvvetlendirme egzersizi': 'diz',
    'hemiparezi': 'tüm vücut',
    'hemiparezi + beyin tümörü': 'tüm vücut',
    'hemipleji': 'tüm vücut',
    'hemipleji yatakta': 'tüm vücut',
    'humerus kırığı': 'omuz',
    'impingement': 'omuz',
    'impingement sendromu': 'omuz',
    'kalça ağrısı': 'kalça',
    'kalça labrum yırtığı opl': 'kalça',
    'kalça protezi': 'kalça',
    'kalça protezi- erken rehabilitasyonu': 'kalça',
    'kalça protezi - erken rehabilitasyonu': 'kalça',
    'kalkaneal spur': 'ayak bileği',
    'kondromalezi patella': 'diz',
    'kondromalezi patella - 2 - kas kuvvetlendirme egzersizi': 'diz',
    'lenfödem + ekstremite ağrısı': 'tüm vücut',
    'medial epikondilit muscular strain': 'dirsek',
    'medial plantar sinir sıkışması': 'ayak bileği',
    'menisektomi - erken rehabilitasyonu': 'diz',
    'menisküs operasyonu lusu': 'diz',
    'metatarsalji': 'ayak bileği',
    'organ nakli mobilizasyon': 'tüm vücut',
    'paraparezi': 'tüm vücut',
    'parapleji': 'tüm vücut',
    'parapleji - tetrapleji': 'tüm vücut',
    'patella kırığı': 'diz',
    'plantar fasiit': 'ayak bileği',
    'serebral palsi': 'tüm vücut',
    'serebrovasküler olay': 'tüm vücut',
    'skolyoz': 'tüm vücut',
    'subtrokanterik kırık': 'kalça',
    'travmatik beyin yaralanması': 'tüm vücut',
    'yürüme': 'tüm vücut',
    'yürüme eğitimi': 'tüm vücut',
    'yaygın kuvvetsizlik': 'tüm vücut',
    'ön çapraz bağ rehabilitasyonu': 'diz',
    'el rehabilitasyonu': 'el bileği',
    'gonartroz - meniskopati + kalkaneal spur': ' diz, ayak bileği', 
    'hemiparezi + el kontraktürü': 'tüm vücut, el bileği', 
    'pediatrik rehabilitasyon programı': 'tüm vücut',
}

def is_list_filled(lst):
    """Yardımcı fonksiyon: Bir listenin dolu olup olmadığını kontrol eder."""
    if not isinstance(lst, list):
        return False
    # Listenin içindeki tüm elemanları kontrol et, en az bir tane dolu varsa 'True' dön.
    return any(isinstance(item, str) and item.strip() != '' for item in lst)

def tahmin_et_uygulama_yeri_liste_versiyonu(row):
    tedavi = row['tedavi_cleaned']
    uygulama_yeri = row['uygulama_yeri_cleaned']

    # 1. Eğer 'uygulama_yeri' listesi zaten doluysa, orijinal listeyi koru.
    if is_list_filled(uygulama_yeri):
        return uygulama_yeri

    # 2. Eğer 'tedavi_cleaned' boş veya metin değilse, işlem yapma.
    if not isinstance(tedavi, str):
        return np.nan

    tedavi_lower = tedavi.lower()
    tespit_edilen_bolge = None

    # 3. Anahtar kelime haritasını kullanarak vücut bölgesini bul.
    for keyword, bolge in KEYWORD_MAP.items():
        if keyword.lower() in tedavi_lower: 
            tespit_edilen_bolge = bolge
            break
            
    # 4. Eğer bir bölge bulunduysa, yön bilgisine göre BİR LİSTE oluştur.
    
    if tespit_edilen_bolge:
        # Öncelik sırasına göre yönü belirle
        if 'bilateral' in tedavi_lower:
            ayrilmayacak_bolgeler = ('sırt', 'tüm vücut', 'yüz')
            
            if tespit_edilen_bolge in ayrilmayacak_bolgeler:
                return [tespit_edilen_bolge]
            # Eğer tespit edilen bölge birden fazla parçadan oluşuyorsa (virgülle ayrılmışsa)
            elif ',' in tespit_edilen_bolge:
                # 1. Bölgeleri virgülle ayırıp her birinin başındaki/sonundaki boşlukları temizle
                bolgeler = [b.strip() for b in tespit_edilen_bolge.split(',')]
                
                # 2. Her bir bölge için sağ ve sol versiyonlarını yeni bir listeye ekle
                sonuc_listesi = []
                for bolge in bolgeler:
                    sonuc_listesi.append(f"sağ {bolge}")
                    sonuc_listesi.append(f"sol {bolge}")
                return sonuc_listesi
            
            else:
                # Tespit edilen bölge tek parçaysa, normal şekilde devam et
                return [f"sağ {tespit_edilen_bolge}", f"sol {tespit_edilen_bolge}"]

        elif 'sağ' in tedavi_lower:
            # 'sağ' durumu virgülle ayrılmış bölgeler için de çalışmalı
            if ',' in tespit_edilen_bolge:
                 bolgeler = [b.strip() for b in tespit_edilen_bolge.split(',')]
                 return [f"sağ {b}" for b in bolgeler]
            else:
                 return [f"sağ {tespit_edilen_bolge}"]

        elif 'sol' in tedavi_lower:
            # 'sol' durumu virgülle ayrılmış bölgeler için de çalışmalı
            if ',' in tespit_edilen_bolge:
                 bolgeler = [b.strip() for b in tespit_edilen_bolge.split(',')]
                 return [f"sol {b}" for b in bolgeler]
            else:
                 return [f"sol {tespit_edilen_bolge}"]
        else:
            # Yön belirtilmemişse ve birden fazla bölge varsa listeye çevir
            if ',' in tespit_edilen_bolge:
                return [b.strip() for b in tespit_edilen_bolge.split(',')]
            else:
                return [tespit_edilen_bolge]
            
    # 5. Hiçbir anahtar kelime bulunamazsa, orijinal boş değeri koru.
    return np.nan

df_unique['uygulama_yeri_doldurulmus'] = df_unique.apply(tahmin_et_uygulama_yeri_liste_versiyonu, axis=1)


filtre_bos = ~df_unique['uygulama_yeri_cleaned'].apply(is_list_filled)

bos_satirlar = df_unique.loc[filtre_bos, ['tedavi_cleaned', 'uygulama_yeri_doldurulmus']]

print("--- 'uygulama_yeri_cleaned' Kolonu Boş Olan Satırlar ---")
print(bos_satirlar)

In [ ]:
# 'uygulama_yeri_doldurulmus' kolonundaki boş (NaN) satırların sayısını bulma
bos_satir_sayisi = df_unique['uygulama_yeri_doldurulmus'].isna().sum()

# Sonucu ekrana yazdırma
print(f"Doldurulmuş kolondaki toplam boş (NaN) satır sayısı: {bos_satir_sayisi}")

In [ ]:
df_unique.info()

In [ ]:
# --- Adım 2: Hastaları Gruplara Ayırma ---
alt_sinir_seans = 10
ust_sinir_seans = 25

def seans_grubu_ata(seans):
    if seans < alt_sinir_seans:
        return 'Erken Bırakan/Hızlı İyileşen'
    elif seans > ust_sinir_seans:
        return 'Uzun Tedavi/Karmaşık Vaka'
    else:
        return 'Standart Tedavi'

# 'seans_grubu' kolonunu oluşturma
df_unique['seans_grubu'] = df_unique['tedavi_seans_sayisi'].apply(seans_grubu_ata)


# --- Adım 3: Gruplara Göre Tanı ve Bölüm Bilgilerini Sayma ---

# 3.1: "Erken Bırakan/Hızlı İyileşen" grubunu filtrele
erken_birakanlar_df = df_unique[df_unique['seans_grubu'] == 'Erken Bırakan/Hızlı İyileşen']
# Bu grubun tanılarını say
erken_birakan_tanilari = erken_birakanlar_df['tanilar_cleaned'].value_counts()
# Bu grubun geldiği bölümleri say
erken_birakan_bolumleri = erken_birakanlar_df['bolum_cleaned'].value_counts()


# 3.2: "Uzun Tedavi/Karmaşık Vaka" grubunu filtrele
uzun_tedavi_df = df_unique[df_unique['seans_grubu'] == 'Uzun Tedavi/Karmaşık Vaka']
# Bu grubun tanılarını say
uzun_tedavi_tanilari = uzun_tedavi_df['tanilar_cleaned'].value_counts()
# Bu grubun geldiği bölümleri say
uzun_tedavi_bolumleri = uzun_tedavi_df['bolum_cleaned'].value_counts()


# --- Adım 4: Sonuçları Yazdırma ---
print("--- Tedaviyi Erken Bırakan / Hızlı İyileşen Hastalar ---")
print("\n[ Tanı Dağılımı ]")
if erken_birakan_tanilari.empty:
    print("Bu grupta hasta bulunamadı.")
else:
    print(erken_birakan_tanilari)

print("\n[ Geldikleri Bölüm Dağılımı ]")
if erken_birakan_bolumleri.empty:
    print("Bu grupta bölüm bilgisi bulunamadı.")
else:
    print(erken_birakan_bolumleri)


print("\n" + "="*60 + "\n")


print("--- Uzun Tedavi Sürecindeki Hastalar ---")
print("\n[ Tanı Dağılımı ]")
if uzun_tedavi_tanilari.empty:
    print("Bu grupta hasta bulunamadı.")
else:
    print(uzun_tedavi_tanilari)

print("\n[ Geldikleri Bölüm Dağılımı ]")
if uzun_tedavi_bolumleri.empty:
    print("Bu grupta bölüm bilgisi bulunamadı.")
else:
    print(uzun_tedavi_bolumleri)

In [ ]:
df_unique.info()

# Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

#Modele Girmeyecek SADECE en ham ve gereksiz sütunları belirleme
cols_to_drop_before_split = [

    'uyruk',
    'kronik_hastalik',
    'bolum',
    'alerji',
    'tanilar',
    'tedavi_adi',
    'uygulama_yeri',
    'alerji_cleaned',
    'tedavi_cleaned',
    'ayrılmış_tanılar',
    'tanilar_cleaned',
    'seans_grubu',
    'hasta_no'
]

# Hedef değişkenimiz 'seans_suresi_dk'
y = df_unique['seans_suresi_dk']

# Özellikler (Features) olarak kullanılacak sütunlar
# Hedef değişkeni ve silinecekler listesindeki sütunları çıkarıyoruz
X = df_unique.drop(columns=['seans_suresi_dk'] + cols_to_drop_before_split, errors='ignore')


# --- Adım 3: Veri Setini Ayırma ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("--- Veri Seti Boyutları ---")
print(f"X_train (Eğitim Özellikleri): {X_train.shape}")
print(f"y_train (Eğitim Hedefi):      {y_train.shape}")
print(f"X_test (Test Özellikleri):    {X_test.shape}")
print(f"y_test (Test Hedefi):         {y_test.shape}")

print("\n--- X_train'de Kalan Sütunlar (Encoding İçin Hazır) ---")

print(X_train.columns.to_list())

# Encoding

## 1- Cinsiyet & Kan Grubu & Uyruk (One-hot Encoding)

In [ ]:
def one_hot_encode(train_df, test_df, column_name):
    # One-hot encoding uygula
    train_dummies = pd.get_dummies(train_df[column_name], prefix=column_name, dtype=int)
    test_dummies = pd.get_dummies(test_df[column_name], prefix=column_name, dtype=int)
    
    # Yeni oluşturulan dummy sütunları orijinal Df'lere ekle
    train_df = pd.concat([train_df, train_dummies], axis=1)
    test_df = pd.concat([test_df, test_dummies], axis=1)
    
    # Test setinde eksik olabilecek sütunları train setine göre hizala
    test_df = test_df.reindex(columns=train_df.columns, fill_value=0)
    
    # Orijinal sütunu sil
    train_df.drop(column_name, axis=1, inplace=True)
    test_df.drop(column_name, axis=1, inplace=True)
    
    return train_df, test_df

X_train, X_test = one_hot_encode(X_train, X_test, 'cinsiyet')
X_train, X_test = one_hot_encode(X_train, X_test, 'kan_grubu')
X_train, X_test = one_hot_encode(X_train, X_test, 'uyruk_cleaned')

print("One-Hot Encoding sonrası X_train boyutu:", X_train.shape)
X_train.head()

## 2- Kronik Hastalık & Alerji & Tanılar (Multi-Label Binarizer)
Bu sütunlar liste içerdiği için MultiLabelBinarizer kullanacağız.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

def multi_label_encode(train_df, test_df, column_name, prefix):
    """
    MultiLabelBinarizer kullanarak bir sütunu encode eder ve sütun isimlerine
    belirtilen prefix'i ekler.
    """
    mlb = MultiLabelBinarizer()
    
    mlb.fit(train_df[column_name])
    
    new_columns = [f"{prefix}_{cls}" for cls in mlb.classes_]
    
    train_encoded = pd.DataFrame(mlb.transform(train_df[column_name]), columns=new_columns, index=train_df.index)
    test_encoded = pd.DataFrame(mlb.transform(test_df[column_name]), columns=new_columns, index=test_df.index)
    
    train_df = pd.concat([train_df, train_encoded], axis=1)
    test_df = pd.concat([test_df, test_encoded], axis=1)
    
    train_df.drop(column_name, axis=1, inplace=True)
    test_df.drop(column_name, axis=1, inplace=True)
    
    return train_df, test_df

In [ ]:
# Kronik Hastalıklar
X_train, X_test = multi_label_encode(X_train, X_test, 'kronik_hastalik_cleaned', prefix='kronik')

# Alerji Grupları
X_train, X_test = multi_label_encode(X_train, X_test, 'alerji_grup', prefix='alerji')

# Tanı Grupları (SGK)
# Önce string'leri listeye çevirelim
X_train['SGK_Tani_Grubu_List'] = X_train['SGK_Tani_Grubu'].apply(lambda x: [g.strip() for g in x.split(",")] if isinstance(x, str) else [])
X_test['SGK_Tani_Grubu_List'] = X_test['SGK_Tani_Grubu'].apply(lambda x: [g.strip() for g in x.split(",")] if isinstance(x, str) else [])
X_train, X_test = multi_label_encode(X_train, X_test, 'SGK_Tani_Grubu_List', prefix='tani_grup')

# Artık ihtiyaç duyulmayan 'SGK_Tani_Grubu' sütununu da silelim
X_train.drop('SGK_Tani_Grubu', axis=1, inplace=True)
X_test.drop('SGK_Tani_Grubu', axis=1, inplace=True)


print("Multi-Label Encoding sonrası X_train boyutu:", X_train.shape)
# Yeni sütun isimlerini kontrol edelim
print("\nOluşturulan Yeni Sütunlardan Bazıları:")
print(X_train.filter(like='kronik_').columns.to_list())
print(X_train.filter(like='alerji_').columns.to_list())
print(X_train.filter(like='tani_grup_').columns.to_list())
print("Multi-Label Encoding sonrası X_train boyutu:", X_train.shape)
X_train.head()

## 3- Bölüm (One-Hot ve Target Encoding)
Bu sütuna hem One-Hot hem de Target Encoding uygulayalım.

In [ ]:
# --- One-Hot Encoding ---
# Önceki adımda kullandığımız fonksiyon, orijinal sütunu sildiği için burada manuel olarak yapacağız.
# Böylece 'bolum_cleaned' sütununu Target Encoding için de kullanabiliriz.

# Eğitim ve test setleri için dummy değişkenleri oluştur
train_dummies = pd.get_dummies(X_train['bolum_cleaned'], prefix='bolum', dtype=int)
test_dummies = pd.get_dummies(X_test['bolum_cleaned'], prefix='bolum', dtype=int)

# Yeni oluşturulan dummy sütunları orijinal Df'lere ekle
X_train = pd.concat([X_train, train_dummies], axis=1)
X_test = pd.concat([X_test, test_dummies], axis=1)


# Adım 1: Ortalamayı SADECE EĞİTİM VERİSİNDEN öğren (y_train kullanarak),leakeage önlemk için
bolum_target_mean = X_train.join(y_train).groupby('bolum_cleaned')['seans_suresi_dk'].mean()

X_train['bolum_target_encoded'] = X_train['bolum_cleaned'].map(bolum_target_mean)
X_test['bolum_target_encoded'] = X_test['bolum_cleaned'].map(bolum_target_mean)

X_test['bolum_target_encoded'].fillna(y_train.mean(), inplace=True)

X_train.drop('bolum_cleaned', axis=1, inplace=True)
X_test.drop('bolum_cleaned', axis=1, inplace=True)

X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

print("Bölüm encoding sonrası X_train boyutu:", X_train.shape)
X_train[['bolum_target_encoded']].head()

In [ ]:
X_train.head()

## 4- Uygulama Yeri (Özel Sayım Kodlaması)
Her bir uygulama bölgesinin kaç kez geçtiğini sayarak özellikler oluşturacağız.

In [ ]:
# Adım 1: Olası tüm bölgeleri SADECE EĞİTİM setinden öğrenelim
all_regions = set()
for lst in X_train['uygulama_yeri_doldurulmus'].dropna():
    for item in lst:
        region = item.replace('sağ ', '').replace('sol ', '')
        all_regions.add(region)
all_regions = sorted(list(all_regions))

# Adım 2: Bu bölgelere göre hem train hem de test setinde sayım sütunları oluşturalım
def count_region_feature(df, regions):
    for region in regions:
        col_name = f"uygulama_yeri_{region.replace(' ', '_')}"
        
        def count_region(lst):
            if not isinstance(lst, list): return 0
            count = 0
            for item in lst:
                if region in item:
                    count += 1
            return count
        df[col_name] = df['uygulama_yeri_doldurulmus'].apply(count_region)
    return df

X_train = count_region_feature(X_train, all_regions)
X_test = count_region_feature(X_test, all_regions)

# Orijinal listeyi içeren sütunu silelim
X_train.drop('uygulama_yeri_doldurulmus', axis=1, inplace=True)
X_test.drop('uygulama_yeri_doldurulmus', axis=1, inplace=True)

print("Uygulama Yeri encoding sonrası X_train boyutu:", X_train.shape)
X_train.filter(like="uygulama_yeri_").head()

## 5- Tedavi Adı (BERT Embeddings + HDBSCAN Clustering)

1. Tıbbi veriler için eğitilmiş BERT modeli ile hem train hem test için anlamsal vektörler (embedding) oluşturulur.
2. HDBSCAN kümeleme modeli SADECE train vektörleri üzerinde eğitilir.
3. Eğitilmiş model, test vektörlerinin hangi kümelere ait olduğunu tahmin etmek için kullanılır.

In [ ]:
!pip install -U transformers
!pip install hdbscan

In [ ]:

from transformers import AutoTokenizer, AutoModel
import torch
import hdbscan

# Model ve tokenizer'ı yükle (bu işlem bir kere yapılır)
model_name = "hazal/BioBERTurkcased-con-trM-trR"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Embedding oluşturma fonksiyonu
def get_bert_embeddings(texts, model, tokenizer, batch_size=16):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size].tolist()
        batch = [str(t) if pd.notna(t) else "" for t in batch]
        
        encodings = tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors="pt", add_special_tokens=True)
        encodings['input_ids'] = encodings['input_ids'].clamp(max=model.config.vocab_size - 1)
        
        with torch.no_grad():
            outputs = model(**encodings)
            all_embeddings.append(outputs.last_hidden_state[:, 0, :]) # CLS token
            
    return torch.cat(all_embeddings, dim=0).cpu().numpy()

# Adım 1: Train ve Test setleri için embedding'leri oluştur
train_embeddings = get_bert_embeddings(X_train['tedavi_cleaned_bert'], model, tokenizer)
test_embeddings = get_bert_embeddings(X_test['tedavi_cleaned_bert'], model, tokenizer)

# Adım 2: HDBSCAN modelini SADECE train embedding'leri ile eğit
clusterer = hdbscan.HDBSCAN(min_cluster_size=8, min_samples=1, metric='euclidean',
                            prediction_data=True) # Tahmin için bu parametre önemli
clusterer.fit(train_embeddings)

# Adım 3: Hem train hem de test setleri için küme etiketlerini al
X_train['tedavi_hdbscan_label'] = clusterer.labels_
X_test['tedavi_hdbscan_label'] = hdbscan.approximate_predict(clusterer, test_embeddings)[0]

# Orijinal metin sütununu silelim
X_train.drop('tedavi_cleaned_bert', axis=1, inplace=True)
X_test.drop('tedavi_cleaned_bert', axis=1, inplace=True)

print("Tedavi Adı kümeleme sonrası X_train boyutu:", X_train.shape)
X_train[['tedavi_hdbscan_label']].head()

# Modellemeye Hazır Veri Setleri
#
Tüm adımlar tamamlandı. `X_train` ve `X_test` artık tamamen sayısal değerlerden oluşuyor, aynı sütunlara sahipler ve herhangi bir veri sızıntısı olmadan oluşturuldular. Verimiz modellemeye hazır.

In [ ]:
print("--- Son X_train Veri Seti Bilgileri ---")
X_train.info()

print("\n--- Son X_test Veri Seti Bilgileri ---")
X_test.info()